In [1]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras.callbacks import ModelCheckpoint
from keras import layers
import pickle
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
os.environ["CUDA_VISIBLE_DEVICES"]="4"


Using TensorFlow backend.


In [2]:
d_feature_vector = pickle.load(open(DATA+"_train_image_caption_feature_vector.p", "rb"))
d_feature_vector_test = pickle.load(open(DATA+"_test_image_caption_feature_vector.p", "rb"))



In [3]:

x_train = []
y_train = []
for i in d_feature_vector.keys():
    x_train.append(d_feature_vector[i])
    label = 1.0
    if i.startswith("neg_"):
        label = 0.0
    y_train.append(label)
x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape)


(3056, 5716)


In [4]:

x_test = []
y_test = []
for i in d_feature_vector_test.keys():
    x_test.append(d_feature_vector_test[i])
    label = 1.0
    if i.startswith("neg_"):
        label = 0.0
    y_test.append(label)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_test.shape)


(768, 5716)


In [5]:
c = 0
for i in y_test:
    if i == 0.0:
        c = c+ 1
print(c)

364


In [9]:
# Input - Layer
model = models.Sequential()
model.add(layers.Dense(1024, activation = "relu", input_dim=5716))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(512, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(256, activation = "relu"))

# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              5854208   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 6,510,593
Trainable params: 6,510,593
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile( optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
checkpoint = ModelCheckpoint(DATA+'-img-caption-model-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  


In [11]:
results = model.fit( x_train,y_train, epochs= 10, batch_size = 32,callbacks=[checkpoint], validation_data = (x_test, y_test))


Train on 3056 samples, validate on 768 samples
Epoch 1/10
3056/3056 [==============================] - 7s 2ms/step - loss: 0.5573 - acc: 0.7117 - val_loss: 0.4868 - val_acc: 0.7656

Epoch 00001: val_loss improved from inf to 0.48678, saving model to JNCA-img-caption-model-best.h5
Epoch 2/10
3056/3056 [==============================] - 1s 260us/step - loss: 0.3676 - acc: 0.8190 - val_loss: 0.5536 - val_acc: 0.7630

Epoch 00002: val_loss did not improve from 0.48678
Epoch 3/10
3056/3056 [==============================] - 1s 238us/step - loss: 0.2668 - acc: 0.8557 - val_loss: 0.7059 - val_acc: 0.7708

Epoch 00003: val_loss did not improve from 0.48678
Epoch 4/10
3056/3056 [==============================] - 1s 229us/step - loss: 0.2499 - acc: 0.8619 - val_loss: 0.9726 - val_acc: 0.7669

Epoch 00004: val_loss did not improve from 0.48678
Epoch 5/10
3056/3056 [==============================] - 1s 229us/step - loss: 0.2452 - acc: 0.8639 - val_loss: 0.9447 - val_acc: 0.7682

Epoch 00005: val_l

In [12]:
model.load_weights(DATA+'-img-caption-model-best.h5', by_name = True)
predicted =  model.predict(x_test)
for i in range(len(predicted)):
    if predicted[i] >= 0.5:
        predicted[i] = 1.0
    else :
        predicted[i] = 0.0

In [13]:
print("fscore",f1_score(y_test, predicted, average="macro"))
print("precision",precision_score(y_test, predicted, average="macro"))
print("recall",recall_score(y_test, predicted, average="macro"))    

x = classification_report(y_test,predicted)
print(x)

fscore 0.7656106940120857
precision 0.768619702956806
recall 0.7681155478185181
             precision    recall  f1-score   support

        0.0       0.72      0.82      0.77       364
        1.0       0.81      0.72      0.76       404

avg / total       0.77      0.77      0.77       768

